In [ ]:
import json
from pprint import pprint
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import MessagesState
from IPython.display import Image, display
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


_ = load_dotenv(find_dotenv())

## Messages

### Message Types

- Human message
- AI message
- System message
- Tool message

In [ ]:
# system message to set the context
messages = [
    SystemMessage(
        content="You are a smart and fun specialist in languages, your job is to help the users find slang and expressions alternatives in other languages. You always responds in a humorous manner."
    ),
    AIMessage(
        content="Hello there, good morning ! What slang do you want to tranlate in which language ?",
        name="Model",
    ),
    HumanMessage(
        content="I would like to find the equivalent expression 'bouncing up' in Portuguese.",
        name="User",
    ),
    AIMessage(
        content="Sure! Here we go, I'm thinking.... searching... a equivalent expression could be: 'dar a volta por cima'",
        name="Model",
    ),
    HumanMessage(
        content="Could you give me more similar expressions ?", name="User"
    ),
]

for msg in messages:
    msg.pretty_print()

In [ ]:
llm_model = ChatOpenAI(model="gpt-4o-mini")
response = llm_model.invoke(messages)

In [ ]:
type(response)

In [ ]:
print(json.dumps(vars(response), indent=4))

## Tools

In [ ]:
def sum_values(a, b):
    """
    Sum two values and return the result.

    Parameters:
        a (float): The first value.
        b (float): The second value.

    Returns:
        float: The sum of a and b.
    """
    return a + b

In [ ]:
llm_model_tool = llm_model.bind_tools([sum_values])

In [ ]:
response_tool = llm_model_tool.invoke(
    [HumanMessage(content="What is 77 added to 3 ?", name="User")]
)
print(json.dumps(vars(response_tool), indent=4))

## Messages + State

In [ ]:
class StateMessageBased(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
messages = [
    SystemMessage(
        content="You are a smart and fun specialist in languages, your job is to help the users find slang and expressions alternatives in other languages. You always responds in a humorous manner."
    )
]
new_message = AIMessage(
    content="Hello there, good morning ! What slang do you want to tranlate in which language ?",
    name="Model",
)

add_messages(messages, new_message)

In [ ]:
class StateMessageBased(MessagesState):
    pass

In [ ]:
# node
def llm_with_tool(state: StateMessageBased):
    return {"messages": [llm_model_tool.invoke(state["messages"])]}

In [ ]:
builder = StateGraph(StateMessageBased)
builder.add_node("llm_with_tool", llm_with_tool)
builder.add_edge(START, "llm_with_tool")
builder.add_edge("llm_with_tool", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke(
    {"messages": HumanMessage(content="Hello there, who are you ?")}
)

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke(
    {"messages": HumanMessage(content="Ok how are you today ??")}
)

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke(
    {"messages": HumanMessage(content="What is 77 added to 3 ?")}
)

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response["messages"][0]

In [ ]:
response["messages"][1]

In [ ]:
response

# Routers

In [ ]:
def mock_tracker_api_call(package_id: str) -> str:
    """
    Fetches an extended package delivery report, including current location, days ongoing and estimated remaining delivery time
    given a package tracking id.

    Parameters:
        package_id (str): The ID of the package to be tracked.

    Returns:
        str: A detailed report of the current package delivery status as a text message.
    """

    response_payload = ()

In [ ]:
mock_tracker_api_call()